In [36]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVR
from sklearn import tree
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import graphviz

In [37]:
flights_df = pd.read_csv('complete_flight_info_and weather_data.csv')

# convert flight date to date object
flights_df = flights_df.drop(['date', 'date.1','CRS_DEP_TIME','ORIGIN','DEST'], axis=1)

flights_df = flights_df.dropna()
 
flights_df['FL_DATE'] = pd.to_datetime(flights_df['FL_DATE'])
del flights_df['DOT_CODE']
# Remove columns starting with 'origin' and 'dest'
columns_to_remove = [col for col in flights_df.columns if col.startswith('ORIGIN') or col.startswith('DEST')]
flights_df = flights_df.drop(columns=columns_to_remove)

# converts string TRUE/FALSE to boolean
flights_df.replace({'TRUE': True, 'FALSE': False}, inplace=True)

# convert FL_Date to year, month, day
flights_df['FL_YEAR'] = pd.to_datetime(flights_df['FL_DATE']).dt.year
flights_df['FL_MONTH'] = pd.to_datetime(flights_df['FL_DATE']).dt.month
flights_df['FL_DAY'] = pd.to_datetime(flights_df['FL_DATE']).dt.day

# drop original date time
flights_df.drop(columns=['FL_DATE'], inplace=True)

X = flights_df.loc[:, flights_df.columns != 'ARR_DELAY']
y = flights_df['ARR_DELAY']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 156, shuffle=True)

In [ ]:
def backward_subset_selection(X, y, cv=5):
    remaining_features = list(X.columns)
    selected_features = remaining_features[:]
    model = LinearRegression()
    scores = cross_val_score(model, X[selected_features], y, cv=cv, scoring='r2')
    best_score = scores.mean()
    
    while len(selected_features) > 1:
        worst_feature = None
        for feature in selected_features:
            remaining_features.remove(feature)
            scores = cross_val_score(model, X[remaining_features], y, cv=cv, scoring='r2')
            mean_score = scores.mean()
            if mean_score > best_score:
                best_score = mean_score
                worst_feature = feature
            remaining_features.append(feature)
        
        if worst_feature is not None:
            selected_features.remove(worst_feature)
            remaining_features.remove(worst_feature)
        else:
            break
    
    return selected_features

selected_features_backward = backward_subset_selection(X_train, y_train)
print("Selected Features (Backward):", selected_features_backward)

In [ ]:
# Subset X_train to include only the selected features
X_train_subset = X_train[selected_features]

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train_subset, y_train)

# Predict on the training data
y_train_pred = model.predict(X_train_subset)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

print("Training MSE:", mse_train)
print("Training R-squared:", r2_train)
# Calculate MAPE for training set
mape_train = np.mean(np.abs((y_train - y_train_pred) / y_train)) * 100

print("Training MAPE:", mape_train)

In [ ]:
# Subset X_test to include only the selected features
X_test_subset = X_test[selected_features]

# Predict on the testing data
y_test_pred = model.predict(X_test_subset)

# Evaluate the model on testing data
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

print("Testing MSE:", mse_test)
print("Testing R-squared:", r2_test)
# Calculate MAPE for training set
mape_train = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100

print("Test MAPE:", mape_train)

In [ ]:
model = tree.DecisionTreeRegressor(max_depth=2, random_state=156) 
model = model.fit(X_train, y_train)

model_text = tree.export_text(model, feature_names=list(X_train.columns))
print(model_text)

In [ ]:
fi = model.feature_importances_

names = X_train.columns
importance_dict = dict(zip(names, fi))

print("Feature Importance:")
for feature, importance in importance_dict.items():
    print(f"{feature}: {importance}")

In [43]:
preds = model.predict(X_test)
print(mean_squared_error(y_test, preds), r2_score(y_test, preds))


1577.5270404657927 0.8681693985770631


In [ ]:
mse = {'k':[], 'train_mse':[], 'test_mse':[], 'train_r2':[], 'test_r2':[], 'test_MAPE': []}
for k in range(1,30):
    print("Fit with max_depth:", k, end='\r', flush=True)
    
    model = tree.DecisionTreeRegressor(max_depth=k)
    model = model.fit(X_train, y_train)
    preds_train = model.predict(X_train)
    preds_test = model.predict(X_test)

    mse['k'].append(k)
    mse['train_mse'].append(mean_squared_error(y_train, preds_train))
    mse['test_mse'].append(mean_squared_error(y_test, preds_test))
    mse['train_r2'].append(r2_score(y_train, preds_train))
    mse['test_r2'].append(r2_score(y_test, preds_test))
    # Calculate MAPE
    abs_errors = np.abs(y_test - y_test_pred)
    percentage_errors = (abs_errors / y_test) * 100
    mse['test_MAPE'].append(np.mean(percentage_errors))

    
idx = mse['test_mse'].index(min(mse['test_mse']))
print('Depth of the model yielding minimum test MSE is:', mse['k'][idx])
print('Optimized model has MSE:', min(mse['test_mse']), 'Optimized model has R2:', mse['test_r2'][idx], 'Optimized model has MAPE:', mse['test_MAPE'][idx])


In [ ]:
# Initialize Gradient Boosting Regressor
model_gb = GradientBoostingRegressor(random_state=156)

# Fit the model
model_gb.fit(X_train, y_train)

# Make predictions
preds_test = model_gb.predict(X_test)

# Evaluate the model
print(mean_squared_error(y_test, preds_test), r2_score(y_test, preds_test))

# Set up parameters for Grid Search
params = {
    'max_depth': np.arange(1, 30, 5),
    'n_estimators': np.arange(1, 30, 5)
}

# Initialize Grid Search
grid_search = GridSearchCV(estimator=model_gb, param_grid=params, cv=5, n_jobs=-1, verbose=1, scoring='neg_mean_absolute_error', return_train_score=True)

# Fit the Grid Search
grid_search.fit(X_train, y_train)



In [ ]:
# Get the best estimator and show parameters
grid_search.best_params_

In [ ]:
# Evaluate the tuned model

clf_best = grid_search.best_estimator_
y_test_pred = clf_best.predict(X_test)

# Calculate MSE and R-squared
mse = mean_squared_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)

# Calculate MAPE
abs_errors = np.abs(y_test - y_test_pred)
percentage_errors = (abs_errors / y_test) * 100
mape = np.mean(percentage_errors)

print('Hyperparameter tuning of boosting yields MSE and R-squared:', mean_squared_error(y_test, y_test_pred), r2_score(y_test, y_test_pred))
print('MSE:', mse)
print('R-squared:', r2)
print('MAPE:', mape)

In [ ]:
# Create the scatter plot
plt.scatter( y_test_pred, y_test)

plt.ylabel('ARR_DELAY'.capitalize())
plt.yticks([0, 50, 100, 150, 200], ['0', '50', '100', '150', '200'])

plt.xlabel('PREDICT_ARR_DELAY'.capitalize())
plt.xticks([0, 50, 100, 150, 200], ['0', '50', '100', '150', '200'])

plt.xlim([0, 200])
plt.ylim([0, 200])

plt.title('Scatter Plot of ARR_DELAY vs PREDICT_ARR_DELAY')

plt.show()